<a href="https://colab.research.google.com/github/jsansao/dlvc/blob/main/Tarefa7_SegmentacaoPets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tarefa 7 - Segmentação do dataset Pets

Nessa tarefa, vamos observar o funcionamento do método U-net no dataset "Pets".  Notebook retirado do repositório [https://github.com/jakeret/unet](https://github.com/jakeret/unet). 


1. Rode o notebook com as configurações atuais; 
2. Opcional - Modifique a profundidade da rede convolucional (em `layer_depth`) e compare o desempenho das métricas e tempo de treinamento. 



In [1]:
!pip install git+https://github.com/jakeret/unet.git

  Cloning https://github.com/jakeret/unet.git to /tmp/pip-req-build-4dbx0xsk
  Running command git clone -q https://github.com/jakeret/unet.git /tmp/pip-req-build-4dbx0xsk
  Created wheel for unet: filename=unet-0.0.post0.dev81+gf557a51-py2.py3-none-any.whl size=25690 sha256=410661f2056cc7f7e8aff271d87af4411940b95f07c5a70203b9838229a4d71d
  Stored in directory: /tmp/pip-ephem-wheel-cache-kqw_mi32/wheels/22/00/93/ce57529ed355f160088cbb6ce086e55168770913b40cf624ba
Successfully built unet


In [2]:
%load_ext tensorboard
import unet
from unet.datasets import oxford_iiit_pet
from tensorflow.keras import losses, metrics

In [5]:
LEARNING_RATE = 1e-3
unet_model = unet.build_model(*oxford_iiit_pet.IMAGE_SIZE,
                              channels=oxford_iiit_pet.channels,
                              num_classes=oxford_iiit_pet.classes,
                              layer_depth=5,
                              filters_root=64,
                              padding="same"
                              )

unet.finalize_model(unet_model,
                    loss=losses.SparseCategoricalCrossentropy(),
                    metrics=[metrics.SparseCategoricalAccuracy()],
                    auc=False,
                    learning_rate=LEARNING_RATE)


In [3]:
train_dataset, validation_dataset = oxford_iiit_pet.load_data()

Shuffling and writing examples to /root/tensorflow_datasets/oxford_iiit_pet/3.2.0.incompleteY4HNKH/oxford_iiit_pet-train.tfrecord
Shuffling and writing examples to /root/tensorflow_datasets/oxford_iiit_pet/3.2.0.incompleteY4HNKH/oxford_iiit_pet-test.tfrecord
Dataset oxford_iiit_pet downloaded and prepared to /root/tensorflow_datasets/oxford_iiit_pet/3.2.0. Subsequent calls will reuse this data.


In [ ]:
%tensorboard --logdir oxford_iiit_pet

In [6]:
trainer = unet.Trainer(name="oxford_iiit_pet", checkpoint_callback=False)
trainer.fit(unet_model,
            train_dataset,
            validation_dataset,
            epochs=25,
            batch_size=32)

Epoch 1/25
  6/115 [>.............................] - ETA: 1:39 - loss: 1.7929 - sparse_categorical_accuracy: 0.4362 - mean_iou: 0.3015 - dice_coefficient: 0.8848WARNING:tensorflow:Callback method `on_train_batch_end` is slow compared to the batch time (batch time: 0.4010s vs `on_train_batch_end` time: 0.4365s). Check your callbacks.


115/115 [==============================] - 158s 1s/step - loss: 1.0358 - sparse_categorical_accuracy: 0.5262 - mean_iou: 0.3039 - dice_coefficient: 0.8890 - val_loss: 0.9092 - val_sparse_categorical_accuracy: 0.5731 - val_mean_iou: 0.3006 - val_dice_coefficient: 0.8931 - learning_rate: 0.0010
Epoch 2/25
115/115 [==============================] - 125s 1s/step - loss: 0.8491 - sparse_categorical_accuracy: 0.6065 - mean_iou: 0.3016 - dice_coefficient: 0.8890 - val_loss: 0.7503 - val_sparse_categorical_accuracy: 0.6859 - val_mean_iou: 0.3048 - val_dice_coefficient: 0.8931 - learning_rate: 0.0010
Epoch 3/25
115/115 [==============================] - 125s 1s/step - loss: 0.7158 - sparse_categorical_accuracy: 0.7062 - mean_iou: 0.3064 - dice_coefficient: 0.8890 - val_loss: 0.6840 - val_sparse_categorical_accuracy: 0.7152 - val_mean_iou: 0.3097 - val_dice_coefficient: 0.8931 - learning_rate: 0.0010
Epoch 4/25
115/115 [==============================] - 125s 1s/step - loss: 0.6631 - sparse_categ

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt


In [ ]:
rows = 10
fig, axs = plt.subplots(rows, 3, figsize=(8, 30))
for ax, (image, label) in zip(axs, train_dataset.take(rows).batch(1)):
  
  prediction = unet_model.predict(image)
  ax[0].matshow(image[0])
  ax[1].matshow(label[0, ..., 0], cmap="gray")
  ax[2].matshow(prediction[0].argmax(axis=-1), cmap="gray")